In [6]:
import csv
from collections import defaultdict
from fuzzywuzzy import fuzz

# Constants representing the paths to the datasets
offer_retailer = 'offer_retailer.csv'
brand_category= 'brand_category.csv'
categories = 'categories.csv'


In [7]:
class OfferSearchTool:
    def __init__(self):
        self.offers = []
        self.brands = {}
        self.categories = {}
        self.load_datasets()

    def load_datasets(self):
        # Load the offer dataset
        with open(offer_retailer, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.offers.append(row)

        # Load the brand and category datasets
        with open(brand_category, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.brands[row['BRAND']] = row['BRAND_BELONGS_TO_CATEGORY']

        with open(categories, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.categories[row['PRODUCT_CATEGORY']] = row['IS_CHILD_CATEGORY_TO']


    def search(self, query):
        # Perform search using fuzzy matching
        results = defaultdict(float)
        for offer in self.offers:
            brand_match_score = self.score_similarity(query, offer['BRAND'])
            retailer_match_score = self.score_similarity(query, offer['RETAILER'])
            category_match_score = self.score_similarity(query, self.brands.get(offer['BRAND'], ''))
            total_score = brand_match_score + retailer_match_score + category_match_score
            if total_score > 0:
                results[offer['OFFER']] += total_score

        sorted_results = sorted(results.items(), key=lambda item: item[1], reverse=True)
        return sorted_results

    def score_similarity(self, query, data_point):
        # Use fuzzy string matching for the similarity score
        return fuzz.partial_ratio(query.lower(), data_point.lower())

    def run_cli(self):
        # Command-line interface for the search tool
        while True:
            query = input("Enter your search query (type 'exit' to stop): ")
            if query.lower() == 'exit':
                break
            results = self.search(query)
            if not results:
                print("No results found.")
                continue
            for offer, score in results:
                print(f"Offer: {offer}, Score: {score}")


if __name__ == "__main__":
    tool = OfferSearchTool()
    tool.run_cli()


Enter your search query (type 'exit' to stop): diapers
Offer: Reese's Peanut Butter Bar, 6 count, at GIANT OR MARTIN’S, Score: 230.0
Offer: GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger, Score: 230.0
Offer: Reese's Chocolate Cones, 8 count at GIANT OR MARTIN’S, Score: 202.0
Offer: Gorton's, at select retailers, Score: 174.0
Offer: Gorton's at select retailers, Score: 174.0
Offer: Spend $210 at Randalls, Score: 172.0
Offer: Any Randalls receipt, Score: 172.0
Offer: Shop 2 times at Randalls, Score: 172.0
Offer: Shop 2 times at ACME, Score: 157.0
Offer: Spend $90 at ACME, Score: 157.0
Offer: Spend $250 at ACME, Score: 157.0
Offer: Any ACME receipt, Score: 157.0
Offer: Spend $130 at ACME, Score: 157.0
Offer: Bubbies Pickles, Sauerkraut, and Condiments at Whole Foods or Sprouts, Score: 143.0
Offer: Sign up for McAlister's Deli Rewards, tap for details, Score: 143.0
Offer: Children's Astepro® Allergy, 22 count at Dollar General, Score: 143.0
Offer: Adult Astepro® Allergy, 22

Enter your search query (type 'exit' to stop): Huggies
Offer: Reese's Peanut Butter Bar, 6 count, at GIANT OR MARTIN’S, Score: 188.0
Offer: Gorton's, at select retailers, Score: 159.0
Offer: GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger, Score: 157.0
Offer: Reese's Chocolate Cones, 8 count at GIANT OR MARTIN’S, Score: 145.0
Offer: Egglife Egg White Wraps at Aldi, Score: 132.0
Offer: Kradle, select varieties, at Pet Supplies Plus, Score: 131.0
Offer: Gorton's at select retailers, Score: 130.0
Offer: Snuggle® Liquid Fabric Softener, at Walmart, Score: 119.0
Offer: Bubbies Pickles, Sauerkraut, and Condiments at Whole Foods or Sprouts, Score: 115.0
Offer: Sign up for McAlister's Deli Rewards, tap for details, Score: 115.0
Offer: Bubbies Pickles, Sauerkraut, OR Condiments at Whole Foods or Sprouts, Score: 115.0
Offer: General Mills™ snacks, select brands, spend $10, Score: 107.0
Offer: Back to the Roots Grow Seed Starting Pots OR Germination Trays, at Walmart or Target, Sco

Enter your search query (type 'exit' to stop): Target
Offer: GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger, Score: 234.0
Offer: Colgate® Toothpaste AND Colgate® Toothbrush, select varieties at Walmart or Target, Score: 200.0
Offer: Gorton's at select retailers, Score: 199.0
Offer: Beyond Steak™ Plant-Based seared tips, 10 ounce at Target, Score: 183.0
Offer: L'Oréal Paris Hair color, select varieties, spend $19 at Target, Score: 183.0
Offer: Back to the Roots Grow Seed Starting Pots OR Germination Trays, at Walmart or Target, Score: 183.0
Offer: Welch's® Juicefuls® Juicy Fruit Snacks, 14 count, at Target, Score: 183.0
Offer: L'Oréal Paris Excellence Hair Color at Target, Score: 183.0
Offer: Back to the Roots Raised Bed Gardening Kit with Soil, Seeds and Plant Food, at Target, Score: 183.0
Offer: L'Oréal Paris Hair color, select varieties, spend $25 at Target, Score: 183.0
Offer: Beyond Steak™ Plant-Based seared tips, 10 ounce, buy 2 at Target, Score: 183.0
Offer: L'Oré

Enter your search query (type 'exit' to stop): exit
